## Milestone Project Food 101 Vision Big

## Check GPU

In [ ]:
!nvidia-smi -L


!pip install -U -q tf-nightly

# Check TensorFlow version (should be minimum 2.4.0+ but 2.13.0+ is better)
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")

# Add timestamp
import datetime
print(f"Notebook last run (end-to-end): {datetime.datetime.now()}")


In [ ]:
# Get helper functions file
import os

if not os.path.exists("helper_functions.py"):
    !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
    print("[INFO] 'helper_functions.py' already exists, skipping download.")


In [ ]:
# Import series of helper functions for the notebook (we've created/used these in previous notebooks)
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys


In [ ]:
# Get TensorFlow datasets
import tensorflow_datasets as tfds

In [ ]:
# List all available datasets
datasets_list = tfds.list_builders()
print(datasets_list)

In [ ]:
(train_data, test_data), ds_info = tfds.load(name='food101',
                                             split=['train','validation'],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True)

In [ ]:
ds_info.features

In [ ]:
# Get the classnames
class_names = ds_info.features['label'].names
class_names[:10]

## Exploring the food101 data from tfds

To become one with our data we want to find
* Class names
* The shape of our input data (image tensors)
* The datatype of our input data
* What the labels look like (e.g. are they one-hot encoded or are they label encoded?)
* Do the labels match up with the class names?


In [ ]:
# Take one sample of our training data
for image, label in train_data.take(1):
  print(f" Image shape: {image.shape} \nImage datatype: {image.dtype}, \nTarget class from Food101 (tesnor form): {label}, \nClass name (str form): {class_names[label.numpy()]}")

In [ ]:
# What does our image tensor look like?
image

In [ ]:
import tensorflow as tf
tf.reduce_min(image), tf.reduce_max(image)

## Plot an image for tensorflow datasets

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with 10 subplots (2 rows, 5 columns)
plt.figure(figsize=(15, 6))  # Adjust figure size as needed

# Take 10 images from train_data
for i, (image, label) in enumerate(train_data.take(10)):
    # Create subplot - i+1 because subplot indexing starts at 1
    plt.subplot(2, 5, i+1)
    plt.imshow(image)
    plt.title(class_names[label.numpy()])
    plt.axis('off')  # Optional: hide axes

plt.tight_layout()  # Adjust spacing between plots
plt.show()

## Create preproccesing functions for our data

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  # Resize the image before normalizing
  image = tf.image.resize(image, [224, 224])
  # Cast to float32 and normalize to [0, 1]
  image = tf.cast(image, dtype=tf.float32)
  return image, label

In [ ]:
preprocessed_img = normalize_img(image,label)[0]
print(f"Image before preprocessing:\n {image[:2]}..., \nShape:{image.shape}, \nDatatype: {image.dtype}\n")
print(f"Image after preprocessing:\n{preprocessed_img[:2]}...,\nShape: {preprocessed_img.shape}, \nDatatype: {preprocessed_img.dtype}")

## Batch and prepare datasets

We're now going to make our data input pipeline run really fast.

In [ ]:
train_data = train_data.map(map_func=normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=(tf.data.AUTOTUNE))

# Map preproccessing function to test data
test_data = test_data.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
train_data, test_data

## Create modelling callbacks
* ModelCheckpoint callback to save our model's progress after feature extraction.

In [ ]:
# Create tensorboard callback (import from helper functions.py)
from helper_functions import create_tensorboard_callback

checkpoint_path = 'model_checkpoint/cp.weights.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor='val_acc',
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      verbose=0)

## Setup mixed precison training

In [ ]:
# Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [ ]:
mixed_precision.global_policy()

In [ ]:
## Build feature extraction model
from tensorflow.keras import layers

# Create a base model
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Create functional model
inputs = layers.Input(shape=input_shape, name='input_layer')
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype=tf.float32, name='softmax_float32')(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
for i, layer in enumerate(model.layers):
  print(layer, layer.dtype, layer.trainable, layer.dtype_policy)

In [ ]:
# Check the dtype_policy attributes for layers in base_model
for layer in model.layers[1].layers:
    # print(layer, layer.dtype, layer.trainable, layer.dtype_policy)
    if layer.dtype_policy.name == "float32":
      print(layer, layer.dtype, layer.trainable, layer.dtype_policy)

In [ ]:
history_101_food_classses_feature_extract = model.fit(train_data,
          epochs=3,
          steps_per_epoch=len(test_data),
          validation_data=test_data,
          validation_steps=int(0.15 * len(test_data)),
          callbacks=[model_checkpoint])

In [ ]:
results_feature_extracts_model = model.evaluate(test_data)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min',
    verbose=1
)

In [ ]:
base_model.trainable = True

In [ ]:
for layer in base_model.layers[:-10]:
  layer.trainable = False

In [ ]:
initial_epoch = 3
last_epoch = history_101_food_classses_feature_extract.epoch[-1]
last_epoch

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=['accuracy'])

In [ ]:
fine_tune_epochs = initial_epoch + 3
fine_tune_history_1 = model.fit(train_data,
                                epochs=100,
                                validation_data = test_data,
                                validation_steps= int(0.15 * len(test_data)),
                                initial_epoch=last_epoch,
                                callbacks=[early_stopping])

In [ ]:
fine_tune_results = model.evaluate(test_data)